In [2]:
import weaviate
import weaviate.classes as wvc
import os
import requests
import json
from dotenv import load_dotenv
from weaviate.classes.config import Configure, Property, DataType, Tokenization
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

load_dotenv()
openai_api_key = os.getenv("OPENAI_KEY")

client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=weaviate.config.AdditionalConfig(timeout=(60, 180)),
    headers={
        "X-OpenAI-Api-Key": openai_api_key  # Replace with your inference API key
    }
)

In [3]:
path_to_pdf = 'pdf_docs'

documents_text = []

for doc in os.listdir(path_to_pdf):

    doc_path = f'{path_to_pdf}/{doc}'
    loader = PyPDFLoader(doc_path)
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
    docs = text_splitter.split_documents(pages)
    documents_text.append(docs)

documents_text = [item for sublist in documents_text for item in sublist]

In [5]:
client.collections.delete("citizens_info_docs") 
client.collections.create(
    "citizens_info_docs",

    properties=[  
        Property(
            name="source",
            data_type=DataType.TEXT,

        ),
        Property(
            name="page", 
            data_type=DataType.TEXT
        ),

        Property(
            name="content", 
            data_type=DataType.TEXT,
            vectorize_property_name=True,
            tokenization=Tokenization.LOWERCASE
        )
    ],

    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="content", source_properties=['content','source']
        )
    ]
)

In [6]:
document_objs = list()

for d in documents_text:
    # Extract data from each document
    source = d.metadata['source']
    page = str(d.metadata['page'])  # page number to string
    content = d.page_content

    # Prepare properties for the data object
    document_objs.append({
        "source": source,
        "page": page,
        "content": content
    })

# Assuming you have a collection named 'Documents' set up in Weaviate
documents = client.collections.get("citizens_info_docs")
documents.data.insert_many(document_objs)  # Insert all documents at once

BatchObjectReturn(all_responses=[UUID('2a017cea-8500-49c5-8a5e-e0b1481cdf55'), UUID('3ffa1047-d73d-4981-b4ff-c0a71a80e222'), UUID('2dcebc88-69ce-45ec-86fb-0d3bcf1f8074'), UUID('3c1552e4-e1a7-4062-94c3-fca8c1fb4b5e'), UUID('0404430e-02d7-4a4d-ba4c-b23fc7136242'), UUID('dc51acfa-ed4d-458d-b5e5-07ce928085ff'), UUID('395a8506-6019-45f6-a55f-c8d74bb00b57'), UUID('9ce7e02b-a089-406b-bad1-61278288cf9d'), UUID('fd1693c1-e4f9-4d84-8a3e-53f12f045e8e'), UUID('13c2d408-98c6-4eb4-84d5-00e8b3d4729f'), UUID('c9c63858-c42b-4662-985b-ca239326dec4'), UUID('e95c4cd0-bc67-4fc9-9e07-f3becc76c96b'), UUID('9a6b92cb-1e53-41e6-8694-8e7b230c74f3'), UUID('9f240f89-835e-4489-964f-7ca81fca1963'), UUID('d1bdb960-3791-46f1-8c28-e1c3565c9191'), UUID('df849aa5-6437-4059-a798-b3712f676300'), UUID('623f8e2b-19ad-41ab-8825-18f8a4b99920'), UUID('49f1896c-3bff-4ecc-a387-9b7f55ff3d17'), UUID('eac0a338-9523-4d03-9cff-8341019193b5')], elapsed_seconds=0.8493316173553467, errors={}, uuids={0: UUID('2a017cea-8500-49c5-8a5e-e0b14

In [7]:
response = documents.query.near_text(
    query="How should one apply for Maternity Benefit and what are the deadlines?",
    limit=1
)

for o in response.objects:
    print(o.properties['content'])  # Object properties

4/12/24, 2:46 PM Pregnancy and social welfare payments
https://www.citizensinformation.ie/en/social-welfare/families-and-children/pregnancy-and-social-welfare-payments/#5011b8 3/3If you are taking part in an employment scheme you have the full range of statutory employment
rights including maternity leave.
If you become pregnant while on an employment scheme (Community Employment, Rural Social
Scheme, Tús and Gateway schemes) you are en tled to take 26 weeks’ maternity leave including an
addi onal 16 weeks of op onal unpaid leave. If possible, you will be supported to complete your 52
weeks of paid work on the same or another project when you ﬁnish your maternity leave.
You should discuss with your sponsor the payment you qualify for while you are on maternity leave.
You may qualify for Maternity Beneﬁt if you have enough PRSI contribu ons. Otherwise, if your
spouse or partner is ge ng a social welfare payment, you may get a qualiﬁed adult increase on their
payment.
Page edited: 21 Sep

In [8]:
# Assuming 'client' has been properly initialized and connected to Weaviate
response = documents.query.near_text(
    query="How to apply for social walfare",
    limit=2
)

# Printing properties of the objects retrieved
for o in response.objects:
    print(o.properties['content'])  # Object properties

4/16/24, 10:04 PM Claiming a social welfare payment
https://www.citizensinformation.ie/en/social-welfare/irish-social-welfare-system/claiming-a-social-welfare-payment/applying-for-a-social-welfare-p… 1/4Claiming a social welfare payment
Introduc on
Rules
How to apply
Where to apply
Further informa on
Introduc on
The Department of Social Protec on has a range of social welfare payments that provide ﬁnancial
support. To get a social welfare payment, you must apply for it. So, if you think you are en tled to a
payment you should always apply.
Your ﬁrst step before applying, is to ﬁnd out which payment you may be en tled to.
Payments are available for:
People who are unemployed
Families and children
Widows, widowers and surviving civil partners
Guardians or orphans
Older and re red people
People with a disability
Carers
If you get a social welfare payment you may also qualify for extra social welfare beneﬁts.
Household Budget Scheme
If you get certain social welfare payments, you can put a

In [9]:
# Assuming 'client' has been properly initialized and connected to Weaviate
response = documents.query.near_text(
    query="What ways do I have to move stuff from the UK",
    limit=2
)

# Printing properties of the objects retrieved
for o in response.objects:
    print(o.properties['content'])  # Object properties

4/17/24, 8:43 AM Shipping your personal belongings back to Ireland
https://www.citizensinformation.ie/en/returning-to-ireland/planning-your-journey-home/shipping-your-belongings-back-to-ireland/ 4/5If you are travelling from the UK to Ireland with your belongings via passenger ferry, go to customs
when you arrive at the port.
Freight ferry
If shipping your belongings by freight ferry, you must give the ferry company your ‘Pre-Boarding
No ﬁca on (PBN) ID’. You can get your PBN ID from Revenue – simply request it when sending your
completed ‘Transfer of residence’ form. See ‘Moving to Ireland from outside the EU’ above.
Read more in Revenue’s guide to the procedure at importa on.
How do I bring my car to Ireland?
If you bring your vehicle to Ireland, you must register the vehicle at a Na onal Car Tes ng Service
(NCTS) centre. You may also have to pay Vehicle Registra on Tax (VRT).
If you’re impor ng a car or small van, the amount of VRT you pay is based on a percentage of its
recommended

In [10]:
# Assuming 'client' has been properly initialized and connected to Weaviate
response = documents.query.near_text(
    query="What do I do if my neighbour is having a party",
    limit=2
)

# Printing properties of the objects retrieved
for o in response.objects:
    print(o.properties['content'])  # Object properties

If your pet is distressed by ﬁreworks, dogstrust.ie have ﬁrework advice and resources available.
Noise from homes
Domes c noise is noise from a domes c se ng, like a home. Depending on the cause of the noise, you will
complain in a diﬀerent way. This is outlined below.
If the noise is coming from a person’s home, the Gardaí can ask them to lower noise but they can’t enter the
property with the sole inten on of asking them to lower the noise.
If the noise con nues, you can complain to the District Court (see above).
There is useful informa on in the leaﬂet on neighbour disputes (pdf) published by FLAC, the Free Legal Advice
Centres.
Noise from rented proper es
If there is noise coming from a rented home, you should approach the tenant ﬁrst and ask them to lower the
noise. If this doesn’t work, you can complain to the landlord. This may be a:
Private landlord
Local authority
Housing associa on
People who are ren ng from private landlords or housing associa ons have certain obliga ons. Th

In [ ]:
# query_return = documents.query.fetch_objects(
#     include_vector=True, 
#     limit=2)